In [1]:
import numpy as np
import pandas as pd

In [15]:
lenses = pd.read_csv('lenses.csv')

In [16]:
from scipy.spatial.distance import squareform, pdist

In [17]:
from scipy.sparse.csgraph import minimum_spanning_tree

In [21]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt

In [77]:
import sys

In [88]:
from sklearn.metrics.pairwise import pairwise_distances

In [137]:
def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

In [250]:
def menorCluster(clusters):
    mySet = set(clusters)
    minimo = len(clusters)
    menor = len(clusters)
    for i in mySet:
        if((clusters.count(i)) < minimo):
            minimo = clusters.count(i)
            menor = i
            
    myList = []
    
    for j in range(len(clusters)):
        if(clusters[j]==menor):
            myList.append(j)
            
    return myList

In [245]:
menorCluster([1,1,2,2,2,3,3,3,4,4,4])

[0, 1]

In [345]:
def genieLinkage(data, limiar):
    
    clusters = {}
    row_index = 0
    col_index = 0
    array = []
    

    for n in range(data.shape[0]):
        array.append(n)
    
        
    clusters[0] = array.copy()

    #finding minimum value from the distance matrix
    #note that this loop will always return minimum value from bottom triangle of matrix
    for k in range(1, data.shape[0]):
        min_val = sys.maxsize
        
        if(gini(clusters[k-1]) <= limiar):
            for i in range(0, data.shape[0]):
                for j in range(0, data.shape[1]):
                    if(data[i][j]<=min_val):
                        min_val = data[i][j]
                        row_index = i
                        col_index = j
        else:
            iList = menorCluster(clusters[k-1])
            for i in iList:
                for j in range(0, data.shape[0]):
                    if(j not in iList):
                        if(data[i][j]<=min_val):
                            min_val = data[i][j]
                            row_index = i
                            col_index = j

        
        #once we find the minimum value, we need to update the distance matrix
        #updating the matrix by calculating the new distances from the cluster to all points
        
        #for Single Linkage
        
#         for i in range(0,data.shape[0]):
#             if(i != col_index):
#                 #we calculate the distance of every data point from newly formed cluster and update the matrix.
#                 temp = min(data[i][col_index],data[row_index][i])
#                 #we update the matrix symmetrically as our distance matrix should always be symmetric
#                 data[row_index][i] = temp
#                 data[i][col_index] = temp

                   
        #set the rows and columns for the cluster with higher index i.e. the row index to infinity
        #Set input[row_index][for_all_i] = infinity
        #set input[for_all_i][row_index] = infinity
#         data[row_index][col_index] = sys.maxsize
#         data[col_index][row_index] = sys.maxsize
            
        #Manipulating the dictionary to keep track of cluster formation in each step
        #if k=0,then all datapoints are clusters
        array = clusters[k-1].copy()
        minimum = min(array[row_index],array[col_index])
        maximum = max(array[row_index],array[col_index])
        
        for n in range(len(array)):
            if(array[n]==maximum):
                array[n] = minimum
        clusters[k] = array.copy()
        
        for i in range(len(array)):
            for j in range(len(array)):
                if(array[i]==array[j]):
                    data[i][j] = sys.maxsize
                    
        print(array)
    return clusters

In [346]:
def genie(dataset, limiar):
    
    dataX = dataset.loc[:,dataset.columns[:-1]].values
    dataY = dataset.loc[:,[dataset.columns[-1]]].values
    
#     dataDistances = pd.DataFrame(squareform(pdist(dataX.iloc[:, 1:])), columns=dataX.index.unique(), index=dataX.index.unique())
    
    dataDistances = pairwise_distances(dataX,metric='euclidean')
    #making all the diagonal elements infinity 
    np.fill_diagonal(dataDistances,sys.maxsize)
    
#     dataMST = minimum_spanning_tree(dataDistances)
    
#     dataMST = dataMST.toarray().astype(float)
    
    Z = genieLinkage(dataDistances, limiar)
#     fig = plt.figure(figsize=(25, 10))
#     dn = dendrogram(Z)
#     plt.show()
    return Z
ourDistances = genie(lenses, 0.36)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 22]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 20, 22, 22]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18, 20, 20, 22, 22]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 16, 18, 18, 20, 20, 22, 22]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 14, 16, 16, 18, 18, 20, 20, 22, 22]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 12, 14, 14, 16, 16, 18, 18, 20, 20, 22, 22]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 12, 12, 14, 14, 16, 16, 18, 18, 20, 20, 22, 22]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 10, 10, 12, 12, 14, 14, 16, 16, 18, 18, 20, 20, 22, 22]
[0, 1, 2, 3, 4, 5, 6, 6, 8, 8, 10, 10, 12, 12, 14, 14, 16, 16, 18, 18, 20, 20, 22, 22]
[0, 1, 2, 3, 4, 4, 6, 6, 8, 8, 10, 10, 12, 12, 14, 14, 16, 16, 18, 18, 20, 20, 22, 22]
[0, 1, 2, 2, 4, 4, 6, 6, 8, 8, 10, 10, 12, 12, 14, 14, 16, 16, 18, 18, 20, 20, 22, 22]
[0, 0, 2, 2, 4, 4, 6, 6, 8, 8, 10, 10, 12, 

In [338]:
ourDistances

{0: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23],
 1: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  22],
 2: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  20,
  22,
  22],
 3: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  18,
  20,
  20,
  22,
  22],
 4: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  16,
  18,
  18,
  20,
  20,
  22,
  22],
 5: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  14,
  16,
  16,
  18,
  18,
  20,
  20,
  22,
  22],
 6: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  12,
  14,
  14,
  16,
  16,
  18,
  18,
  20,
  20,
  22,
  22],
 7: [0,
  1,
  2,
  3,
  4,
  5,
 

In [347]:
for i in range(len(ourDistances)):
    print(gini(ourDistances[i]))

0.347222222222
0.345
0.343369829684
0.342338217338
0.341911764706
0.342097170972
0.342901234568
0.344330855019
0.346393034826
0.349094881398
0.352443609023
0.356446540881
0.361111111111
0.378205128205
0.390625
0.398148148148
0.400537634409
0.397540983607
0.388888888889
0.452380952381
0.474358974359
0.444444444444
0.666666666667
nan


/Users/ajgan/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [165]:
gini([2,1,1,1])

0.14999999999999999

In [190]:
myArray = [1,1,2,3]
myArray.count(1)

2